In [ ]:
!pip install -qU diffusers transformers accelerate invisible-watermark>=0.2.0

# Stable Diffusion XL

Compared with previous Stable Diffusion models, in the Stable Diffusion XL:
1. the UNet is 3 times larger and SDXL combines a second text encoder (OpenCLIP ViT-bigG/14) with the original text encoder to significantly increase the number of parameters
2. introduces size and crop-conditioning to preserve training data from being discarded and gain more control over how a generated image should be cropped
3. introduces a two-stage model process; the *base* model (can also be run as a standalone model) generates an image as an input to the *refiner* model which adds additional high-quality details

## Model checkpoints

In [ ]:
from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline
import torch

pipeline = StableDiffusionXLPipeline.from_pretrained(
    'stabilityai/stable-diffusion-xl-base-1.0',
    torch_dtype=torch.float16,
    variant='fp16',
    use_safetensors=True
).to('cuda')

refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    'stabilityai/stable-diffusion-xl-refiner-1.0',
    torch_dtype=torch.float16,
    variant='fp16',
    use_safetensors=True
).to('cuda')

We can also use the `from_single_file()` method

In [ ]:
from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline
import torch

pipeline = StableDiffusionXLPipeline.from_single_file(
    "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/blob/main/sd_xl_base_1.0.safetensors",
    torch_dtype=torch.float16
).to("cuda")

refiner = StableDiffusionXLImg2ImgPipeline.from_single_file(
    "https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/blob/main/sd_xl_refiner_1.0.safetensors",
    torch_dtype=torch.float16
).to("cuda")

## Text-to-image

SDXL generates a 1024x1024 image for the best results. Anything below 512x512 may not work.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline_text2image = AutoPipelineForText2Image.from_pretrained(
    'stabilityai/stable-diffusion-xl-base-1.0',
    torch_dtype=torch.float16,
    variant='fp16',
    use_safetensors=True
).to('cuda')

In [ ]:
prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"
image = pipeline_text2image(prompt=prompt).images[0]
image

Output hidden; open in https://colab.research.google.com to view.

## Image-to-image

SDXL works well with image size between 768x768 and 1024x1024 for image-to-image task.

In [ ]:
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import load_image, make_image_grid

# use `from_pipe` to avoid consuming additional memory when loading a checkpoint
pipeline_image2image = AutoPipelineForImage2Image.from_pipe(pipeline_text2image).to('cuda')

In [ ]:
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/sdxl-text2img.png"
init_image = load_image(url)
prompt = "a dog chatching a firsbee in the jungle"

image = pipeline_image2image(
    prompt,
    image=init_image,
    strength=0.8,
    guidance_scale=10.5
).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

Output hidden; open in https://colab.research.google.com to view.

## Inpainting

In [ ]:
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

# use `from_pipe` to avoid consuming additional memory when loading a checkpoint
pipeline_inpaint = AutoPipelineForInpainting.from_pipe(pipeline_text2image).to('cuda')

In [ ]:
img_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/sdxl-text2img.png"
mask_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/sdxl-inpaint-mask.png"

init_image = load_image(img_url)
mask_image = load_image(mask_url)
prompt = "a deep sea diver floating"

image = pipeline_inpaint(
    prompt,
    image=init_image,
    mask_image=mask_image,
    strength=0.85,
    guidance_scale=12.5
).images[0]
make_image_grid([init_image, mask_image, image], rows=1, cols=3)

Output hidden; open in https://colab.research.google.com to view.

## Refine image quality

There are two ways to use the refiner:
1. use the base and refiner models together to produce a refined image
2. use the base model to produce an image, and subsequently use the refiner model to add more details to the image (this is how SDXL was originally trained)

### Base + refiner model

This is known as an **ensemble of expert denoisers**.

The *ensemble of expert denoisers* approach requires fewer overall denosing steps versus passing the base model's output to the refiner model, so it should be significantly faster to run. However, we cannot inspect the base model'output because it still contains a large amount of noise.

As an ensemble of expert denoisers, the base model serves as the expert during the high-noise diffusion stage and the refiner model serves as the expert during the low-noise diffusion stage.

In [ ]:
from diffusers import DiffusionPipeline
import torch

base = DiffusionPipeline.from_pretrained(
    'stabilityai/stable-diffusion-xl-base-1.0',
    torch_dtype=torch.float16,
    variant='fp16',
    use_safetensors=True
).to('cuda')

refiner = DiffusionPipeline.from_pretrained(
    'stabilityai/stable-diffusion-xl-refiner-1.0',
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    variant='fp16',
    use_safetensors=True
).to('cuda')

To use this approach, we need to define the number of timesteps for each model to run through their respective stages.
* For the base model, this is controlled by the `denoising_end` parameter,
* For the refiner model, this is controlled by the `denoising_start` parameter

Here, we set `denoising_end=0.8` so the base model performs the first 80% of denoising the **high-noise** timesteps and set `denoising_start=0.8` so the refiner model performs the last 20% of denoising the **low-noise** timesteps. The base model output should be in **latent** space instead of a PIL image.

In [ ]:
prompt = "a majestic lion jumping from a big stone at night"

image_base = base(
    prompt,
    num_inference_steps=40,
    denoising_end=0.8,
    output_type='latent'
).images

image_refined = refiner(
    prompt,
    num_inference_steps=40,
    denoising_start=0.8,
    image=image_base
).images[0]
image_refined

Output hidden; open in https://colab.research.google.com to view.

The refiner model can also be used for inpainting in the `StableDiffusionXLInpaintPipeline`:

In [ ]:
from diffusers import StableDiffusionXLInpaintPipeline
from diffusers.utils import load_image, make_image_grid
import torch

base = StableDiffusionXLInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

refiner = StableDiffusionXLInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
).to("cuda")

In [ ]:
img_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png"
mask_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo_mask.png"

init_image = load_image(img_url)
mask_image = load_image(mask_url)
prompt = "a majestic tiger sitting on a bench"

num_inference_steps= 75
high_noise_frac = 0.7

image = base(
    prompt=prompt,
    image=init_image,
    mask_image=mask_image,
    num_inference_steps=num_inference_steps,
    denoising_end=high_noise_frac,
    output_type="latent",
).images
image = refiner(
    prompt=prompt,
    image=image,
    mask_image=mask_image,
    num_inference_steps=num_inference_steps,
    denoising_start=high_noise_frac,
).images[0]
make_image_grid([init_image, mask_image, image.resize((512, 512))], rows=1, cols=3)

This ensemble of expert denoisers method works well for all available schedulers.

### Base to refiner model

SDXL gets a boost in image quality by using the refiner model to add additional high-quality details to the fully-denoised image from the base model, in an image-to-image setting.

In [ ]:
from diffusers import DiffusionPipeline
from diffusers.utils import make_image_grid
import torch

base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
).to("cuda")

We can use SDXL refiner with a different base model. For example, we can use the `Hunyuan-DiT` or `PixArt-Sigma` pipelines to generate images with better prompt adhernce and then pass it to the SDXL refiner to enhance final generation quality.

In [ ]:
prompt = "astronaut in a jungle, cold color palette, muted colors, detailed, 8k"

image = base(
    prompt,
    output_type='latent'
).images[0]

image = refiner(
    prompt,
    image=image[None, :]
).images[0]
image

Output hidden; open in https://colab.research.google.com to view.

## Micro-conditioning

The *micro-conditioning* includes original image size, target image size, and cropping parameters. The micro-conditionings can be used at inference time to create high-quality, centered images.

### Size conditioning

* `original_size` conditioning comes from upscaled images in the training batch. During inference, we can use `original_size` to indicate the original image resolution. Using the default value of `(1024, 1024)` produces higher-quality images that resemble the 1024x1024 images in the dataset.
* `target_size` conditioning comes from finetuning SDXL to support different image aspect ratio. During inference, if we use the default value of `(1024, 1024)`, we will get an image that resembles the composition of square images in the dataset.

In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch

pipe = StableDiffusionXLPipeline.from_pretrained(
    'stabilityai/stable-diffusion-xl-base-1.0',
    torch_dtype=torch.float16,
    variant='fp16',
    use_safetensors=True
).to('cuda')

In [2]:
from diffusers.utils import make_image_grid

prompt = "astronaut in a jungle, cold color palette, muted colors, detailed, 8k"

image128 = pipe(
    prompt,
    negative_original_size=(128, 128),
    negative_target_size=(1024, 1024),
).images[0]
image256 = pipe(
    prompt,
    negative_original_size=(256, 256),
    negative_target_size=(1024, 1024),
).images[0]
image512 = pipe(
    prompt,
    negative_original_size=(512, 512),
    negative_target_size=(1024, 1024),
).images[0]
image1024 = pipe(
    prompt,
    negative_original_size=(1024, 1024),
    negative_target_size=(1024, 1024),
).images[0]
make_image_grid([image128, image256, image512, image1024], rows=1, cols=4)

Output hidden; open in https://colab.research.google.com to view.

In [3]:
image128 = pipe(
    prompt,
    negative_original_size=(1024, 1024),
    negative_target_size=(128, 128),
).images[0]
image256 = pipe(
    prompt,
    negative_original_size=(1024, 1024),
    negative_target_size=(256, 256),
).images[0]
image512 = pipe(
    prompt,
    negative_original_size=(1024, 1024),
    negative_target_size=(512, 512),
).images[0]
image1024 = pipe(
    prompt,
    negative_original_size=(1024, 1024),
    negative_target_size=(1024, 1024),
).images[0]
make_image_grid([image128, image256, image512, image1024], rows=1, cols=4)

Output hidden; open in https://colab.research.google.com to view.

### Crop conditioning

Images generated by SDXL may appear to be cropped, because images are actually cropped during trianing so that all the images in a batch have the same size. By conditioning on crop coordinates, SDXL learns that no cropping - coordinates `(0, 0)` - usually correlates with centered subjects and complete faces.

In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch

pipe = StableDiffusionXLPipeline.from_pretrained(
    'stabilityai/stable-diffusion-xl-base-1.0',
    torch_dtype=torch.float16,
    variant='fp16',
    use_safetensors=True
).to('cuda')

In [4]:
prompt = "astronaut in a jungle, cold color palette, muted colors, detailed, 8k"

image = pipe(
    prompt,
    crops_coords_top_left=(256, 0)
).images[0]
image

Output hidden; open in https://colab.research.google.com to view.

We can also specify negative cropping coordinates to steer generation away from certain cropping parameter

In [5]:
image = pipe(
    prompt,
    negative_original_size=(512, 512),
    negative_crops_coords_top_left=(0,0),
    negative_target_size=(1024, 1024),
).images[0]
image

Output hidden; open in https://colab.research.google.com to view.

## Use a different prompt for each text-encoder

SDXL uses two text-encoders, so it is possible to pass a different prompt to each text-encoder, which can improve quality.

In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch

pipe = StableDiffusionXLPipeline.from_pretrained(
    'stabilityai/stable-diffusion-xl-base-1.0',
    torch_dtype=torch.float16,
    variant='fp16',
    use_safetensors=True
).to('cuda')

In [6]:
# prompt is passed to OpenAI CLIP-ViT/L-14
prompt = "astronaut in a jungle, cold color palette, muted colors, detailed, 8k"

# prompt is passed to OpenCLIP-ViT/bigG-14
prompt_2 = "Van Goph painting"

image = pipe(
    prompt,
    prompt_2=prompt_2
).images[0]
image

Output hidden; open in https://colab.research.google.com to view.

In [7]:
image = pipe(
    prompt_2,
    prompt_2=prompt,
).images[0]
image

Output hidden; open in https://colab.research.google.com to view.